In [50]:
import scipy.io as sio
import numpy as np
import math
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from scipy.fftpack import fft,ifft

## Load the dataset

In [3]:
u_bar_dict = sio.loadmat('u_bar_region_13.mat')
u_bar_dict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'u_bar'])

In [4]:
u_bar_dict['u_bar'].shape

(128, 1250000)

In [51]:
all_input_data = u_bar_dict['u_bar'].transpose()

In [52]:
pi_region = sio.loadmat('PI_region_13.mat')
pi_region.keys()

dict_keys(['__header__', '__version__', '__globals__', 'PI'])

In [53]:
pi_region['PI'].shape

(128, 1250000)

In [54]:
all_output_data = pi_region['PI'].transpose()

In [55]:
# training will be in the first 100k samples only
training_samples = 100000
input_data = all_input_data[:training_samples]
output_data = all_output_data[:training_samples]

In [75]:
def data_normalize(data):
    '''
    Normalize the data to have zero mean and unit variance
    '''
    data_mean = np.mean(data, axis=0)
    data_std = np.std(data, axis=0)
    data_normalized = (data - data_mean) / data_std
    return data_normalized, data_mean.reshape(-1,1), data_std.reshape(-1,1)

input_data_normalized, input_mean, input_std = data_normalize(input_data)
output_data_normalized, output_mean, output_std = data_normalize(output_data)

In [15]:
# Split the data into training and testing sets
training_input, testing_input, training_output, testing_output = train_test_split(input_data_normalized, 
                                                                                  output_data_normalized, 
                                                                                  test_size=0.5, 
                                                                                  random_state=42, 
                                                                                  shuffle=True)

In [39]:
def swish(x):
    '''
    Swish activation function
    '''
    return x * keras.activations.sigmoid(x)


In [40]:
def get_model():
    '''
    Create the neural network model
    The architecture of the model is the same defined in the paper 
    It's a feedforward neural network with 6 hidden layers, each with 250 neurons
    '''
    model = Sequential()
    model.add(Dense(128,input_shape=(128,), activation=swish))
    model.add(Dropout(0.2))
    model.add(Dense(250, activation=swish))
    model.add(Dropout(0.2))
    model.add(Dense(250, activation=swish))
    model.add(Dropout(0.2))
    model.add(Dense(250, activation=swish))
    model.add(Dropout(0.2))
    model.add(Dense(250, activation=swish))
    model.add(Dropout(0.2))
    model.add(Dense(250, activation=swish))
    model.add(Dropout(0.2))
    model.add(Dense(250, activation=swish))
    model.add(Dropout(0.2))
    model.add(Dense(128,activation=None))

    return model

model = get_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


/home/deadlypoison/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
model.fit(training_input, training_output, epochs=100, batch_size=128, validation_data=(testing_input, testing_output))

Epoch 1/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 0.9283 - mae: 0.4568 - val_loss: 0.6591 - val_mae: 0.3702
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 0.6480 - mae: 0.3748 - val_loss: 0.4499 - val_mae: 0.2705
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.5084 - mae: 0.3238 - val_loss: 0.4081 - val_mae: 0.2499
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.4565 - mae: 0.2977 - val_loss: 0.3728 - val_mae: 0.2276
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.4310 - mae: 0.2813 - val_loss: 0.3660 - val_mae: 0.2223
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.4090 - mae: 0.2702 - val_loss: 0.3375 - val_mae: 0.2080
Epoch 7/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.3918 - mae: 0.2607 - val_loss: 0.3476 - val_mae: 0.2111
Epoch 8/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.3778 - mae: 0.2515 - val_loss: 0.3115 - val_mae: 0.1926
Epoch 9/100
391/391 ━━━━━━━━━━━━━━━━━━━━

In [83]:
model.save('./trained_models/model_13.h5')

In [84]:
# Some constants from the paper implementation
s = 20 # the scale of the timstep
dt = s*1e-2 # the timestep of the simulation
NX = 128 # the number of grid points in the x direction
Lx = 100 # the length of the domain
nu = 2e-2 # the viscosity

dx = Lx/NX # the grid spacing
x = np.linspace(0, Lx-dx, NX) # the grid points

kx = (2*math.pi/Lx)*np.concatenate((np.arange(0, NX/2+1,dtype=float),np.arange((-NX/2+1),0,dtype=float))).reshape([NX,1]) # from the paper implementation

In [85]:
number_iterations = 100000

D1 = 1j*kx
D2 = kx*kx
D1 = D1.reshape([NX, 1])
D2 = D2.reshape([NX, 1])
D2_tensor = np.float32(
    (D2[0:int(NX/2)]-np.mean(D2[0:int(NX/2)])/np.std(D2[0:int(NX/2)])))

D2x = 1 + 0.5*dt*nu*D2

In [86]:
u_store = np.zeros([NX, number_iterations]) # store the velocity field
sub_store = np.zeros([NX, number_iterations]) # store the predicted SGS term from the neural network

region = 13
pred_start = training_samples + 50000 # start the prediction from the 150k sample

In [87]:
force_dict = sio.loadmat('f_bar_all_regions.mat')
force_dict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'f_bar'])

In [88]:
force_bar = force_dict['f_bar'][:, int((region-1)*12500)+int(pred_start/s):]

In [89]:
u_old = all_input_data[pred_start-1, :].reshape([NX, 1])
u = all_input_data[pred_start, :].reshape([NX, 1])
u_fft = fft(u, axis=0)
u_old_fft = fft(u_old, axis=0)

In [90]:
def normalize(data, mean, std):
    return (data - mean) / std

def denormalize(data, mean, std):
    return data * std + mean

In [91]:
prev_subgrid = model.predict(normalize(u,input_mean,input_std).reshape([1,128])).reshape([128,1])
prev_subgrid = denormalize(prev_subgrid, output_mean, output_std)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


In [ ]:
for i in range(number_iterations):
  subgrid = model.predict(normalize(u,input_mean,input_std).reshape([1,128])).reshape([128,1])
  subgrid = denormalize(subgrid, output_mean, output_std)

  force = force_bar[:, i].reshape((NX, 1))

  F = D1*fft(.5*(u**2), axis=0)
  F0 = D1*fft(.5*(u_old**2), axis=0)

  uRHS = -0.5*dt*(3*F - F0) - 0.5*dt*nu*(D2*u_fft) + u_fft + dt*fft(force, axis=0) \
      - fft(dt*3/2*subgrid + 1/2*dt*prev_subgrid, axis=0)

  prev_subgrid = subgrid
  u_old_fft = u_fft
  u_old = u

  u_fft = uRHS/D2x.reshape([NX, 1])
  u = np.real(ifft(u_fft, axis=0))
  u_store[:, i] = u.squeeze()
  sub_store[:, i] = subgrid.squeeze()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

/home/deadlypoison/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/array_slicing.py:63: RuntimeWarning: overflow encountered in cast
  return x.astype(dtype)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

In [ ]:
sio.savemat('./DDP_results',
           {'u_pred': u_store, 'sub_pred':sub_store})